The experiment is done with a random split so we should not detect a data drift

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import xgboost as xgb
import os
import sys

# TODO: handle this by running setup.py ?
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from src.drift.drift_explainer import DriftExplainer

# Breast Cancer Data

In [ ]:
dataset = datasets.load_breast_cancer()

In [ ]:
X = pd.DataFrame(dataset.data, columns = dataset.feature_names)
y = dataset.target

In [ ]:
X.head()

# Build XGBoost model

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

In [ ]:
clf = XGBClassifier(n_estimators=1000,
                    booster="gbtree",
                    objective="binary:logistic",
                    learning_rate=0.05,
                    max_depth=6,
                    use_label_encoder=False,
                    seed=2021)

In [ ]:
clf.fit(X=X_train, y=y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=20, verbose=10)

In [ ]:
#clf.save_model('models/xgboost_binary_classif_breast_cancer')

In [ ]:
#clf = XGBClassifier()
#clf.load_model('models/xgboost_binary_classif_breast_cancer')

# Analyze data drift

In [ ]:
drift_explainer = DriftExplainer()

In [ ]:
drift_explainer.fit(clf, X1=X_train, X2=X_valid, y1=y_train, y2=y_valid)

In [ ]:
drift_explainer.plot_prediction_drift()

In [ ]:
drift_explainer.get_prediction_drift()

In [ ]:
drift_explainer.plot_prediction_drift(prediction_type='proba')

In [ ]:
drift_explainer.get_prediction_drift(prediction_type='proba')

In [ ]:
drift_explainer.plot_feature_contribs(type='size_diff')

In [ ]:
drift_explainer.get_feature_contribs(type='size_diff')

In [ ]:
drift_explainer.plot_feature_contribs(type='mean_diff')

In [ ]:
drift_explainer.get_feature_contribs(type='mean_diff')

In [ ]:
drift_explainer.plot_target_drift()

In [ ]:
drift_explainer.get_target_drift()

In [ ]:
drift_explainer.plot_feature_drift(0)

In [ ]:
drift_explainer.get_feature_drifts()

# With base XGBoost API

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

In [ ]:
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_valid, y_valid)

In [ ]:
param = {'max_depth': 6,
         'learning_rate': 0.05,
         'objective': 'binary:logistic',
         'booster': 'gbtree',
         'seed': 2021
        }

In [ ]:
bst = xgb.train(param, dtrain, num_boost_round=1000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=20, verbose_eval=10)

In [ ]:
drift_explainer = DriftExplainer()
drift_explainer.fit(bst, X1=X_train, X2=X_valid, y1=y_train, y2=y_valid)